In [7]:
import pandas as pd

df = pd.read_csv("../data/processed/player_match_summary.csv")
df.head()


,match_id,player,team,runs,balls,fours,sixes,dismissed,runs_conceded,balls_bowled,wicket,overs
0,1082591,A Choudhary,Royal Challengers Bangalore,6.0,2.0,0.0,1.0,0.0,55.0,25.0,1.0,4.1
1,1082591,A Nehra,Sunrisers Hyderabad,0.0,0.0,0.0,0.0,0.0,42.0,24.0,2.0,4.0
2,1082591,B Kumar,Sunrisers Hyderabad,0.0,0.0,0.0,0.0,0.0,28.0,24.0,2.0,4.0
3,1082591,BCJ Cutting,Sunrisers Hyderabad,16.0,6.0,0.0,2.0,0.0,35.0,22.0,1.0,3.4
4,1082591,Bipul Sharma,Sunrisers Hyderabad,0.0,0.0,0.0,0.0,0.0,4.0,6.0,1.0,1.0


In [8]:
bat_stats = df.groupby("player").agg(
    innings=("runs", "count"),
    runs=("runs", "sum"),
    balls=("balls", "sum"),
    fours=("fours", "sum"),
    sixes=("sixes", "sum"),
    dismissals=("dismissed", "sum")
).reset_index()

bat_stats["average"] = bat_stats.apply(
    lambda x: x["runs"] / x["dismissals"] if x["dismissals"] > 0 else x["runs"], axis=1
)

bat_stats["strike_rate"] = 100 * bat_stats["runs"] / bat_stats["balls"]
bat_stats["boundary_rate"] = (bat_stats["fours"] + bat_stats["sixes"]) / bat_stats["balls"]

bat_stats.fillna(0, inplace=True)
bat_stats.head(10)


,player,innings,runs,balls,fours,sixes,dismissals,average,strike_rate,boundary_rate
0,A Ashish Reddy,28,280.0,193.0,16.0,15.0,15.0,18.666667,145.077720,0.160622
1,A Badoni,48,963.0,695.0,73.0,38.0,36.0,26.750000,138.561151,0.159712
2,A Chandila,12,4.0,7.0,0.0,0.0,0.0,4.000000,57.142857,0.000000
3,A Chopra,6,53.0,71.0,7.0,0.0,6.0,8.833333,74.647887,0.098592
4,A Choudhary,5,25.0,20.0,1.0,1.0,1.0,25.000000,125.000000,0.100000
5,A Dananjaya,1,4.0,5.0,0.0,0.0,0.0,4.000000,80.000000,0.000000
6,A Flintoff,3,62.0,53.0,5.0,2.0,2.0,31.000000,116.981132,0.132075
7,A Kamboj,11,16.0,14.0,2.0,0.0,2.0,8.000000,114.285714,0.142857
8,A Kumble,42,35.0,47.0,3.0,0.0,3.0,11.666667,74.468085,0.063830
9,A Manohar,20,292.0,235.0,23.0,14.0,19.0,15.368421,124.255319,0.157447


In [9]:
bowl_stats = df.groupby("player").agg(
    balls_bowled=("balls_bowled", "sum"),
    runs_conceded=("runs_conceded", "sum"),
    wickets=("wicket", "sum")
).reset_index()

bowl_stats["overs"] = bowl_stats["balls_bowled"] // 6 + (bowl_stats["balls_bowled"] % 6) / 10

bowl_stats["economy"] = bowl_stats["runs_conceded"] / (bowl_stats["balls_bowled"] / 6)
bowl_stats["bowling_avg"] = bowl_stats["runs_conceded"] / bowl_stats["wickets"].replace(0, 1)
bowl_stats["bowling_strike_rate"] = bowl_stats["balls_bowled"] / bowl_stats["wickets"].replace(0, 1)

bowl_stats.fillna(0, inplace=True)
bowl_stats.head(10)


,player,balls_bowled,runs_conceded,wickets,overs,economy,bowling_avg,bowling_strike_rate
0,A Ashish Reddy,264.0,400.0,19.0,44.0,9.090909,21.052632,13.894737
1,A Badoni,35.0,50.0,4.0,5.5,8.571429,12.500000,8.750000
2,A Chandila,234.0,245.0,11.0,39.0,6.282051,22.272727,21.272727
3,A Chopra,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,A Choudhary,102.0,144.0,5.0,17.0,8.470588,28.800000,20.400000
5,A Dananjaya,24.0,47.0,0.0,4.0,11.750000,47.000000,24.000000
6,A Flintoff,66.0,106.0,2.0,11.0,9.636364,53.000000,33.000000
7,A Kamboj,192.0,288.0,10.0,32.0,9.000000,28.800000,19.200000
8,A Kumble,970.0,1089.0,49.0,161.4,6.736082,22.224490,19.795918
9,A Manohar,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [10]:
career_stats = pd.merge(bat_stats, bowl_stats, on="player", how="outer")
career_stats.fillna(0, inplace=True)
career_stats.head(20)


,player,innings,runs,balls,fours,sixes,dismissals,average,strike_rate,boundary_rate,balls_bowled,runs_conceded,wickets,overs,economy,bowling_avg,bowling_strike_rate
0,A Ashish Reddy,28,280.0,193.0,16.0,15.0,15.0,18.666667,145.077720,0.160622,264.0,400.0,19.0,44.0,9.090909,21.052632,13.894737
1,A Badoni,48,963.0,695.0,73.0,38.0,36.0,26.750000,138.561151,0.159712,35.0,50.0,4.0,5.5,8.571429,12.500000,8.750000
2,A Chandila,12,4.0,7.0,0.0,0.0,0.0,4.000000,57.142857,0.000000,234.0,245.0,11.0,39.0,6.282051,22.272727,21.272727
3,A Chopra,6,53.0,71.0,7.0,0.0,6.0,8.833333,74.647887,0.098592,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,A Choudhary,5,25.0,20.0,1.0,1.0,1.0,25.000000,125.000000,0.100000,102.0,144.0,5.0,17.0,8.470588,28.800000,20.400000
5,A Dananjaya,1,4.0,5.0,0.0,0.0,0.0,4.000000,80.000000,0.000000,24.0,47.0,0.0,4.0,11.750000,47.000000,24.000000
6,A Flintoff,3,62.0,53.0,5.0,2.0,2.0,31.000000,116.981132,0.132075,66.0,106.0,2.0,11.0,9.636364,53.000000,33.000000
7,A Kamboj,11,16.0,14.0,2.0,0.0,2.0,8.000000,114.285714,0.142857,192.0,288.0,10.0,32.0,9.000000,28.800000,19.200000
8,A Kumble,42,35.0,47.0,3.0,0.0,3.0,11.666667,74.468085,0.063830,970.0,1089.0,49.0,161.4,6.736082,22.224490,19.795918
9,A Manohar,20,292.0,235.0,23.0,14.0,19.0,15.368421,124.255319,0.157447,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [11]:
career_stats["batting_strength"] = (
    0.4 * career_stats["average"] +
    0.3 * career_stats["strike_rate"] +
    0.2 * career_stats["boundary_rate"] * 100 +
    0.1 * (career_stats["innings"] - career_stats["dismissals"])
)

career_stats["bowling_strength"] = (
    0.5 * career_stats["wickets"] -
    0.3 * career_stats["economy"] +
    0.2 * (1 / career_stats["bowling_strike_rate"].replace(0, 1))
)

career_stats["overall_strength"] = (
    career_stats["batting_strength"] + career_stats["bowling_strength"]
)


In [12]:
career_stats.to_csv("../data/processed/player_career_stats.csv", index=False)
print("Saved player_career_stats.csv!")


Saved player_career_stats.csv!


In [14]:
df_sorted = df.sort_values(["player", "match_id"])

# Last 5-match rolling averages using transform
df_sorted["recent_runs"] = (
    df_sorted.groupby("player")["runs"]
    .transform(lambda x: x.rolling(5, min_periods=1).mean())
)

df_sorted["recent_balls"] = (
    df_sorted.groupby("player")["balls"]
    .transform(lambda x: x.rolling(5, min_periods=1).mean())
)

df_sorted["recent_wickets"] = (
    df_sorted.groupby("player")["wicket"]
    .transform(lambda x: x.rolling(5, min_periods=1).mean())
)

# For economy: sum of last 5 balls / sum of last 5 overs
df_sorted["recent_runs_conceded"] = (
    df_sorted.groupby("player")["runs_conceded"]
    .transform(lambda x: x.rolling(5, min_periods=1).sum())
)

df_sorted["recent_balls_bowled"] = (
    df_sorted.groupby("player")["balls_bowled"]
    .transform(lambda x: x.rolling(5, min_periods=1).sum())
)

df_sorted["recent_economy"] = df_sorted["recent_runs_conceded"] / (
    df_sorted["recent_balls_bowled"] / 6
)

df_sorted["recent_economy"].replace([float('inf'), -float('inf')], 0, inplace=True)
df_sorted["recent_economy"].fillna(0, inplace=True)

df_sorted.head(20)


C:\Users\choud\AppData\Local\Temp\ipykernel_17916\1387313131.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sorted["recent_economy"].replace([float('inf'), -float('inf')], 0, inplace=True)
C:\Users\choud\AppData\Local\Temp\ipykernel_17916\1387313131.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting valu

,match_id,player,team,runs,balls,fours,sixes,dismissed,runs_conceded,balls_bowled,wicket,overs,recent_runs,recent_balls,recent_wickets,recent_runs_conceded,recent_balls_bowled,recent_economy
18569,548329,A Ashish Reddy,Deccan Chargers,0.0,0.0,0.0,0.0,0.0,29.0,12.0,0.0,2.0,0.000000,0.000000,0.0,29.0,12.0,14.500000
18772,548341,A Ashish Reddy,Deccan Chargers,0.0,0.0,0.0,0.0,0.0,32.0,24.0,2.0,4.0,0.000000,0.000000,1.0,61.0,36.0,10.166667
18878,548346,A Ashish Reddy,Deccan Chargers,10.0,10.0,0.0,1.0,1.0,11.0,13.0,1.0,2.1,3.333333,3.333333,1.0,72.0,49.0,8.816327
18921,548348,A Ashish Reddy,Deccan Chargers,0.0,0.0,0.0,0.0,0.0,32.0,19.0,1.0,3.1,2.500000,2.500000,1.0,104.0,68.0,9.176471
19009,548352,A Ashish Reddy,Deccan Chargers,3.0,3.0,0.0,0.0,1.0,16.0,13.0,1.0,2.1,2.600000,2.600000,1.0,120.0,81.0,8.888889
19096,548356,A Ashish Reddy,Deccan Chargers,0.0,0.0,0.0,0.0,0.0,36.0,24.0,1.0,4.0,2.600000,2.600000,1.2,127.0,93.0,8.193548
19158,548359,A Ashish Reddy,Deccan Chargers,8.0,8.0,1.0,0.0,1.0,40.0,24.0,2.0,4.0,4.200000,4.200000,1.2,135.0,93.0,8.709677
19449,548373,A Ashish Reddy,Deccan Chargers,10.0,4.0,2.0,0.0,0.0,17.0,12.0,0.0,2.0,4.200000,3.000000,1.0,141.0,92.0,9.195652
19513,548376,A Ashish Reddy,Deccan Chargers,4.0,4.0,0.0,0.0,1.0,25.0,24.0,3.0,4.0,5.000000,3.800000,1.4,134.0,97.0,8.288660
19681,598000,A Ashish Reddy,Sunrisers Hyderabad,7.0,4.0,1.0,0.0,0.0,21.0,12.0,1.0,2.0,5.800000,4.000000,1.4,139.0,96.0,8.687500


In [15]:
df_sorted.to_csv("../data/processed/player_recent_form.csv", index=False)
print("Saved player_recent_form.csv!")


Saved player_recent_form.csv!
